In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
def describe_dish_flavor(input_data):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
You are a professional food analyst.

Your task is to analyze the dish in the image.
DO NOT recommend wine.
Output MUST be valid JSON.
        """),

        HumanMessagePromptTemplate.from_template([
            {
                "text": """
Analyze the dish in the image and output strictly in the following JSON format:

{{
  "dish_name": "",
  "cooking_method": "",
  "flavor_profile": {{
    "saltiness": "low | medium | high",
    "fatness": "low | medium | high",
    "acidity": "low | medium | high",
    "sweetness": "low | medium | high",
    "umami": "low | medium | high"
  }},
  "pairing_keywords": []
}}
"""
            },
            {"image_url": "{image_url}"}
        ])
    ])

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        api_key=GOOGLE_API_KEY,
        temperature=0.1
    )

    chain = prompt | llm | StrOutputParser()
    return chain


In [7]:
from langchain_core.runnables import RunnableLambda
r1 = RunnableLambda(describe_dish_flavor)

input_data ={'image_url': "https://image.8dogam.com/resized/product/asset/v1/upload/aa83c097ff804e5eb732c34d2b94805a.jpg?type=big&res=3x&ext=jpg"}

In [8]:
res = r1.invoke(input_data)
print(res)

```json
{
  "dish_name": "Raw Pork Belly Slices",
  "cooking_method": "Uncooked",
  "flavor_profile": {
    "saltiness": "low",
    "fatness": "high",
    "acidity": "low",
    "sweetness": "low",
    "umami": "medium"
  },
  "pairing_keywords": [
    "Grilled",
    "Braised",
    "Roasted",
    "Soy sauce",
    "Garlic",
    "Ginger",
    "Chili",
    "Kimchi",
    "Vinegar",
    "Sesame oil",
    "Asian spices",
    "Rice",
    "Fresh herbs",
    "Lettuce wraps"
  ]
}
```


In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


In [13]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11204\1474760240.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [15]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("wine-review-v2")

In [16]:
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    text_key="text"   # 저장할 때 쓴 필드명
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)

In [ ]:
# =========================
# 0. ENV & IMPORTS
# =========================
from dotenv import load_dotenv
import os, json

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# LangChain
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# Pinecone & Embeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings


# =========================
# 1. IMAGE → DISH ANALYSIS
# =========================
def describe_dish_flavor(input_data):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
You are a professional food analyst.
Analyze the dish in the image.
DO NOT recommend wine.
Output MUST be valid JSON.
        """),

        HumanMessagePromptTemplate.from_template([
            {
                "text": """
Analyze the dish in the image and output strictly in the following JSON format:

{{
  "dish_name": "",
  "cooking_method": "",
  "flavor_profile": {{
    "saltiness": "low | medium | high",
    "fatness": "low | medium | high",
    "acidity": "low | medium | high",
    "sweetness": "low | medium | high",
    "umami": "low | medium | high"
  }},
  "pairing_keywords": []
}}
"""
            },
            {"image_url": "{image_url}"}
        ])
    ])

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        api_key=GOOGLE_API_KEY,
        temperature=0.1
    )

    return prompt | llm | StrOutputParser()


# =========================
# 2. DISH JSON → SEARCH QUERY
# =========================
def build_wine_query(dish_json: dict) -> str:
    f = dish_json["flavor_profile"]
    return f"""
Dish: {dish_json["dish_name"]}
Cooking method: {dish_json["cooking_method"]}

Flavor profile:
- Saltiness: {f["saltiness"]}
- Fatness: {f["fatness"]}
- Acidity: {f["acidity"]}
- Sweetness: {f["sweetness"]}
- Umami: {f["umami"]}

Keywords: {", ".join(dish_json["pairing_keywords"])}
"""


# =========================
# 3. PINECONE RETRIEVER
# =========================
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("wine-review-v2")

vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    text_key="text"
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)


# =========================
# 4. SOMMELIER RAG PROMPT
# =========================
sommelier_prompt = PromptTemplate.from_template("""
You are a professional wine sommelier.

Dish information:
{dish}

Wine options from our catalog:
{context}

Your task:
- Select the single best wine
- Explain clearly why it pairs well with the dish
- Keep the explanation intuitive and customer-friendly

Rules:
- Do NOT invent wines
- Only recommend from the provided options
""")


llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        api_key=GOOGLE_API_KEY,
        temperature=0.1)


# =========================
# 5. FULL RAG EXECUTION
# =========================
def run_wine_rag(image_url: str):
    # 1️⃣ Dish analysis chain 생성
    dish_chain = describe_dish_flavor(None)

    # 2️⃣ 실행
    dish_result = dish_chain.invoke({"image_url": image_url})

    # 3️⃣ JSON 파싱
    dish_json = safe_json_loads(dish_result)

    # 4️⃣ 검색 쿼리 생성
    query_text = build_wine_query(dish_json)

    # 5️⃣ 벡터 검색
    docs = retriever.invoke(query_text)

    # 6️⃣ 소믈리에 응답
    answer = sommelier_chain.invoke({
        "dish": query_text,
        "context": "\n\n".join(d.page_content for d in docs)
    })

    return {
        "dish_analysis": dish_json,
        "recommended_wine": answer
    }



# =========================
# 6. RUN
# =========================
image_url = "https://image.8dogam.com/resized/product/asset/v1/upload/aa83c097ff804e5eb732c34d2b94805a.jpg?type=big&res=3x&ext=jpg"

result = run_wine_rag(image_url)

print("🍽️ Dish analysis:")
print(result["dish_analysis"])
print("\n🍷 Wine recommendation:")
print(result["recommended_wine"])


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
print("RAW dish_result:")
print(dish_result)
print("TYPE:", type(dish_result))


RAW dish_result:


NameError: name 'dish_result' is not defined